# Regression: Predict Bullet Drop

This will use table data from popular open-source ballistic algorithms to train a regression model to predict bullet drop in mils given: range, bullet weight, bullet length, "G" scale, BC, barrel twist, and muzzle velocity

In [ ]:
# Use seaborn for pairplot.
!pip install -q seaborn

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

### Data
Using pandas, import data.

In [ ]:
url = 'http://github.com/shembree89/data/ballistics.data'
column_names = ['Drop', 'Range', 'Weight', 'Length', 'G', 'BC', 'MV', 'Twist']

raw_dataset = pd.read_csv(url, names=column_names, sep=' ', skipinitialspace=True)

In [ ]:
dataset = raw_dataset.copy()
dataset.tail()

### Split the data into training and test sets

Will use 80% of the data for training and 20% to test.

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

### Inspect the data

See if there is a correlation (we know there is)

In [ ]:
sns.pairplot(train_dataset[['Drop', 'Range', 'Weight', 'Length', 'G', 'BC', 'MV', 'Twist']], diag_kind='kde')

Overall Statistics

In [ ]:
train_dataset.describe().transpose()

### Split features from labels

Label is the target value, the value the model will be trained to predict.

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('Drop')
test_labels = test_features.pop('Drop')

## Normalization

The range of values of the different columns are very different. For "Range" we have anywhere between 0 - 1k meters, for "MV" we have values in the tens of thousands, but BC is single digits. For the weights to work properly we need all values to be in similar ranges while still representing their relative value. However, this means that during actual use of the model later, the input values will also have to be normalized the same way.

In [ ]:
train_dataset.describe().transpose()[['mean', 'std']]

In [ ]:
# create the layer
normalizer = tf.keras.layers.Normalization(axis=-1)

In [ ]:
# fit preprocessing layer state to the data
normalizer.adapt(np.array(train_features))

In [ ]:
# calculate the mean/variance and store in layer
print(normalizer.mean.numpy())

When the layer is called, it returns the input data, with each feature independently normalized:

In [ ]:
first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

## Regression

In [ ]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

In [ ]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

In [ ]:
%%time
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

In [ ]:
plot_loss(history)

Collect the results on the test set:

In [ ]:
test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)

## Performance

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [Drop]']).T

### Make predictions

In [ ]:
test_predictions = dnn_model.predict(test_features).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [Drop]')
plt.ylabel('Predictions [Drop]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)


Error distribution:

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [Drop]')
_ = plt.ylabel('Count')

Save model

In [ ]:
dnn_model.save('dnn_model.keras')

Test reloading model

In [ ]:
reloaded = tf.keras.models.load_model('dnn_model.keras')

test_results['reloaded'] = reloaded.evaluate(
    test_features, test_labels, verbose=0)

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [Drop]']).T